# Yelp Data Challenge - Restaurant Recommender

Lei Chen

Nov 2018

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

In [2]:
df = pd.read_csv('../data/last_2_years_restaurant_reviews.csv', encoding='ISO-8859-1' )

In [3]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,address,...,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,avg_stars,state
0,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
1,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
2,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-08-22,1,1nK5w0VNfDlnR3bOz13dJQ,5,My husband and I went there for lunch on a Sat...,1,gm8nNoA3uB4In5o_Hxpq3g,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
3,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
4,--9e1ONYQuAa-CB_Rrw7Tw,0,2015-02-02,0,_Uwp6FO1X-avE9wqTMC59w,5,This place is first class in every way. Lobste...,0,-Z7Nw2UF7NiBSAzfXNA_XA,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV


## 1. Clean data and get rating data 

#### Select relevant columns in the original dataframe

In [9]:
df_bus = df[['business_id', 'name']].rename(columns={'name':'business_name'})
df_bus.head()

,business_id,business_name
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse


In [12]:
bus_dic = {}
for _, row in df_bus.iterrows():
    bus_dic[row.business_id] = row.business_name
bus_dic

{'--9e1ONYQuAa-CB_Rrw7Tw': 'Delmonico Steakhouse',
 '-1m9o3vGRA8IBPNvNqKLmA': "Bavette's Steakhouse & Bar",
 '-1vfRrlnNnNJ5boOVghMPA': 'Red Ginseng Narita Sushi & BBQ',
 '-3zffZUHoY8bQjGfPSoBKQ': 'Michael Mina',
 '-8R_-EkGpUhBk55K9Dd4mg': 'Sin City Thai Restaurant',
 '-9YyInW1wapzdNZrhQJ9dg': 'Fresh Buffet',
 '-AD5PiuJHgdUcAK-Vxao2A': 'Smashburger',
 '-ADtl9bLp8wNqYX1k3KuxA': 'Ichi Ramen House',
 '-AGdGGCeTS-njB_8GkUmjQ': 'Woo Chun Korean BBQ',
 '-BS4aZAQm9u41YnB9MUASA': 'Argana',
 '-Bf8BQ3yMk8U2f45r2DRKw': 'In-N-Out Burger',
 '-BmqghX1sv7sgsxOIS2yAg': 'Merkato Ethiopian Cafe',
 '-Bv-HHUs8aHzDrdWcZHn8w': 'Smashburger',
 '-C8sSrFqaCxp51pyo-fQLQ': 'The Rice Shop',
 '-CQokjildrY7UZezXCdEBw': 'Toddy Shop',
 '-CbDQXiuKzPQ0_jiUz03aw': 'Taiga Modern Japanese & Thai Restaurant',
 '-EohS3nXZGl6odE9Lx5BSw': 'Panda Express',
 '-FNquqGseSCVMWo7KbK-Tg': 'Chipotle Mexican Grill',
 '-FcZY7a7qgxTUlTvwuyJnQ': '8 Noodle Bar',
 '-G7MPSNBpxRJmtrJxdwt7A': 'La Costa Del Sol',
 '-IWsoxH7mLJTTpU5MmWY4w': 'Pho

In [13]:
import pickle
pickle.dump(bus_dic, open("../data/business.pkl", 'wb'))

In [14]:
# Get business_id, user_id, stars for recommender

df_rec = df[['business_id', 'user_id', 'stars']]
df_rec.head()

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,ymlnR8UeFvB4FZL56tCZsA,5
1,--9e1ONYQuAa-CB_Rrw7Tw,9pSSL6X6lFpY3FCRLEH3og,5
2,--9e1ONYQuAa-CB_Rrw7Tw,gm8nNoA3uB4In5o_Hxpq3g,5
3,--9e1ONYQuAa-CB_Rrw7Tw,CEtidlXNyQzgJSdF1ubPFw,3
4,--9e1ONYQuAa-CB_Rrw7Tw,-Z7Nw2UF7NiBSAzfXNA_XA,5


In [15]:
df_rec['timestamp']=1

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_rec.head()

,business_id,user_id,stars,timestamp
0,--9e1ONYQuAa-CB_Rrw7Tw,ymlnR8UeFvB4FZL56tCZsA,5,1
1,--9e1ONYQuAa-CB_Rrw7Tw,9pSSL6X6lFpY3FCRLEH3og,5,1
2,--9e1ONYQuAa-CB_Rrw7Tw,gm8nNoA3uB4In5o_Hxpq3g,5,1
3,--9e1ONYQuAa-CB_Rrw7Tw,CEtidlXNyQzgJSdF1ubPFw,3,1
4,--9e1ONYQuAa-CB_Rrw7Tw,-Z7Nw2UF7NiBSAzfXNA_XA,5,1


In [17]:
from surprise import KNNBaseline, Reader
from surprise import Dataset

In [18]:
reader = Reader(rating_scale=(1, 5))
bus_data = Dataset.load_from_df(df_rec[['user_id', 'business_id', 'stars']], reader)

In [19]:
bus_data

In [20]:
print('build similarity matrix..')
trainset = bus_data.build_full_trainset()

build similarity matrix..


In [21]:
trainset

In [22]:
trainset.n_items

5086

In [23]:
trainset.n_users

276675

#### Find the neighbour user

In [28]:
lucky_user = df_rec.user_id.values[67]
lucky_user

'zO9kz3Y48I9s2BrLFg2ymA'

In [29]:
print('Start training model..')
algo = KNNBaseline()
algo.train(trainset)

lucky_user = df_rec.user_id.values[67]
print('This is our lucky user: ', lucky_user)

# get neighbourhoods
# get inner_id
user_inner_id = algo.trainset.to_inner_uid(lucky_user)
print('inner id: ', user_inner_id)

user_neighbors = algo.get_neighbors(user_inner_id, k=15)

# convert the neighbors back from inner id to origin id
user_neighbors = (algo.trainset.to_raw_uid(inner_id)
                 for inner_id in user_neighbors)

print()
print('15 Users nearest to user ', lucky_user, " are")
for user in user_neighbors:
    print(user)

Start training model..
Estimating biases using als...


/opt/anaconda/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Computing the msd similarity matrix...


MemoryError: 

### Predict via NMF

In [30]:
from surprise import NMF, evaluate

algo = NMF()
trainset = bus_data.build_full_trainset()
algo.train(trainset)

In [31]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for business in items:
    print(algo.predict(algo.trainset.to_raw_uid(user_inner_id), algo.trainset.to_raw_iid(business), r_ui=1), bus_dic[algo.trainset.to_raw_iid(business)])

user: -Z7Nw2UF7NiBSAzfXNA_XA item: --9e1ONYQuAa-CB_Rrw7Tw r_ui = 1.00   est = 4.94   {'was_impossible': False} Delmonico Steakhouse


### Store model

In [36]:
import surprise
surprise.dump.dump('../model/recommendation.model', algo=algo)
# load
algo = surprise.dump.load('../model/recommendation.model')